In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Imports

In [0]:
import pandas as pd
import numpy as np
import math

## Loading & Preprocessing data

In [0]:
ratings = pd.read_csv("/content/gdrive/My Drive/Recommender system/data/rating.csv", header=None, sep = None)
ratings.rename(columns = {0:"user_id", 1:"movie_id", 2:"rating", 3:"unix_timestamp"}, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
print(ratings.shape)
ratings.head()

(100003, 4)


,user_id,movie_id,rating,unix_timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [0]:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv("/content/gdrive/My Drive/Recommender system/data/item.csv", names = i_cols  ,sep="|", encoding='latin-1')

In [0]:
print(items.shape)
items.head()

(1682, 24)


,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [0]:
ratings.movie_id = ratings.movie_id - 1

In [0]:
items.movie_id = items.movie_id - 1

In [0]:
ratings_train = ratings.iloc[0:80001, :]
ratings_test = ratings.iloc[80001:, :]

In [0]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,0,49,5,881250949
1,0,171,5,881250949
2,0,132,1,881250949
3,196,241,3,881250949
4,186,301,3,891717742


## Implementing Funk SVD

In [0]:
nusers = 944
nmovies = 1682

In [0]:
r = 30  #no of latent features
lrate=0.035 # learning rate
regularizer=0.01 #regulariization
maxepochs = 30 
minimprov = 0.001 #minimum improvement

In [0]:
avg = sum(ratings["rating"])/len(ratings) #average rating of the entire dataset
initval = math.sqrt(avg/r)

In [0]:
U = np.full((nusers,r), initval) #initializing user feature affinity matrix
V = np.full((nmovies,r), initval) #initializing item feature relevance matrix

In [0]:
print(U.shape, V.shape)

(944, 30) (1682, 30)


In [0]:
def rmse(ratings_):
  sse = 0.0
  total = 0
  for i in range(len(ratings_)):
    crating = ratings_.values[i]
    sse += (crating[2] - predict_rating(crating[0], crating[1]))**2
    total += 1
  return math.sqrt(sse/total)

In [0]:
def predict_rating(user_id, item_id):
  r = np.dot(U[user_id], V[item_id])
  if r > 5:
    r = 5
  elif r < 1:    
    r = 1
  return r

In [0]:
def train(k, ratings_train):
  sse = 0.0
  n = 0
  for i in range(len(ratings_train)):#stochatic because different rating used in every step, for every datapoint get optimized coefficient value
      # get current rating
      crating = ratings_train.values[i]
      err = crating[2] - predict_rating(crating[0], crating[1]) #error is calculated for one rating whereas in batch GD error is calculated for all the training examples
      sse += err**2
      n += 1

      uTemp = U[crating[0]][k]
      vTemp = V[crating[1]][k]

      U[crating[0]][k] += lrate * (err*vTemp - regularizer*uTemp)
      V[crating[1]][k] += lrate * (err*uTemp - regularizer*vTemp)
  return math.sqrt(sse/n)

In [0]:
#in case of sgd, one epoch is n updatesto the parameters of gd where n= no. of training examples
#in contrast with batch gradient descent where to calculate the gradient we sum over all the training examples
def trainfeatures(ratings_train):
  ratings_train = ratings_train
  old_train_err = 1000000
  for k in range(r): #r is no. of latent features
    print("k=", k)
    for epoch in range(maxepochs): #one epoch is one entire stochastic descent, an epoch is an iteration 
      train_err = train(k, ratings_train) #train error after SGD
      if abs(old_train_err - train_err) < minimprov:
        break
      old_train_err = train_err
      print( "epoch=", epoch, "; train error=", train_err)

In [0]:
trainfeatures(ratings)

k= 0
epoch= 0 ; train error= 1.0551662551721717
epoch= 1 ; train error= 1.025705497591321
epoch= 2 ; train error= 1.019488832219351
epoch= 3 ; train error= 1.0155493702329068
epoch= 4 ; train error= 1.0124589740292833
epoch= 5 ; train error= 1.0098046502459717
epoch= 6 ; train error= 1.0074746578050489
epoch= 7 ; train error= 1.0054491833063917
epoch= 8 ; train error= 1.0037136014394699
epoch= 9 ; train error= 1.0022497770907872
epoch= 10 ; train error= 1.001012206216037
epoch= 11 ; train error= 0.9999636898553889
k= 1
epoch= 0 ; train error= 0.952079377724045
epoch= 1 ; train error= 0.9250907130640247
epoch= 2 ; train error= 0.9186065810938067
epoch= 3 ; train error= 0.9159157522210992
epoch= 4 ; train error= 0.9145234424197723
k= 2
epoch= 0 ; train error= 0.9037896474213589
epoch= 1 ; train error= 0.899741141050314
epoch= 2 ; train error= 0.8976138217587467
epoch= 3 ; train error= 0.8960468725976215
epoch= 4 ; train error= 0.8947560462977308
epoch= 5 ; train error= 0.8936268205711739

In [0]:
print("Train RMSE: ", rmse(ratings_train))
print("Test RMSE: ", rmse(ratings_test))

Train RMSE:  0.8551505262145315
Test RMSE:  0.8244027776922302


In [0]:
mov_dic

NameError: ignored

In [0]:
pd.DataFrame(R)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.654487,3.238670,3.086853,3.336479,3.212539,3.902400,3.639918,3.677178,3.698078,3.623002,...,3.079552,3.346181,3.346181,3.346181,3.273770,2.859917,3.242867,3.346181,3.346181,3.346181
1,3.902859,2.998330,2.637382,3.506244,2.911268,4.450397,4.002075,4.418358,4.205849,4.040838,...,3.190563,3.597763,3.597763,3.597763,3.448602,2.669394,3.414395,3.597763,3.597763,3.597763
2,3.983244,3.147865,2.656296,3.407559,3.073324,4.299296,3.806215,4.436287,3.858897,3.646431,...,3.261452,3.628188,3.628188,3.628188,3.495770,2.798958,3.459005,3.628188,3.628188,3.628188
3,2.813579,2.596895,2.484613,2.963995,2.472353,2.886793,2.941956,3.279257,3.155546,2.969891,...,2.941225,2.971237,2.971237,2.971237,2.915710,2.628479,2.899916,2.971237,2.971237,2.971237
4,4.831879,4.325873,4.198324,4.161627,4.252092,5.243878,4.792731,4.315179,4.833134,4.765506,...,3.611939,4.087500,4.087500,4.087500,3.999336,3.510213,3.963105,4.087500,4.087500,4.087500
5,3.357650,2.544563,2.381428,3.397746,2.379324,4.036528,3.829892,4.065850,4.276428,4.103328,...,3.003637,3.323668,3.323668,3.323668,3.173861,2.413280,3.150789,3.323668,3.323668,3.323668
6,3.257742,2.567388,2.431395,3.251061,2.555786,3.591639,3.480996,3.921424,3.779920,3.601317,...,3.028844,3.244388,3.244388,3.244388,3.131796,2.449846,3.086235,3.244388,3.244388,3.244388
7,4.572046,3.714162,3.461237,3.871914,3.765077,4.890004,4.432036,4.585094,4.500308,4.333925,...,3.462758,3.929363,3.929363,3.929363,3.808118,3.013833,3.737401,3.929363,3.929363,3.929363
8,4.260261,3.366481,2.908462,3.592850,3.313935,4.714574,4.129758,4.554024,4.172988,4.022190,...,3.311334,3.766535,3.766535,3.766535,3.625975,2.895172,3.589789,3.766535,3.766535,3.766535
9,4.485483,3.809421,3.638512,3.890843,3.794571,4.849488,4.445357,4.332035,4.536916,4.412867,...,3.433642,3.873245,3.873245,3.873245,3.768462,3.121527,3.715256,3.873245,3.873245,3.873245


In [0]:
# Recommendations for User_id = 0 in a sorted manner
preds_0 = preds_df.iloc[0,:].sort_values(ascending=False)

# Recommendations if user has not rated the movie previously
recommendations_0 = [i for i in preds_0.index if df.loc[0,i]==0]

# Recommend only top 10 items
final_recommendations_0 = recommendations_0[:10]
print("Recommendations for user with id 0 are: " + ', '.join(final_recommendations_0))

In [0]:
pred_df = pd.DataFrame(R, columns=list(mov_dic.values()))

In [0]:
pred_df.head()

,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
0,3.654487,3.238670,3.086853,3.336479,3.212539,3.902400,3.639918,3.677178,3.698078,3.623002,...,3.079552,3.346181,3.346181,3.346181,3.273770,2.859917,3.242867,3.346181,3.346181,3.346181
1,3.902859,2.998330,2.637382,3.506244,2.911268,4.450397,4.002075,4.418358,4.205849,4.040838,...,3.190563,3.597763,3.597763,3.597763,3.448602,2.669394,3.414395,3.597763,3.597763,3.597763
2,3.983244,3.147865,2.656296,3.407559,3.073324,4.299296,3.806215,4.436287,3.858897,3.646431,...,3.261452,3.628188,3.628188,3.628188,3.495770,2.798958,3.459005,3.628188,3.628188,3.628188
3,2.813579,2.596895,2.484613,2.963995,2.472353,2.886793,2.941956,3.279257,3.155546,2.969891,...,2.941225,2.971237,2.971237,2.971237,2.915710,2.628479,2.899916,2.971237,2.971237,2.971237
4,4.831879,4.325873,4.198324,4.161627,4.252092,5.243878,4.792731,4.315179,4.833134,4.765506,...,3.611939,4.087500,4.087500,4.087500,3.999336,3.510213,3.963105,4.087500,4.087500,4.087500


In [0]:
pred_df.iloc[62,:].sort_values(ascending=False)

Pather Panchali (1955)                                                         4.767208
Casablanca (1942)                                                              4.679831
Wallace & Gromit: The Best of Aardman Animation (1996)                         4.622629
Close Shave, A (1995)                                                          4.599144
One Flew Over the Cuckoo's Nest (1975)                                         4.583588
Usual Suspects, The (1995)                                                     4.572977
Shawshank Redemption, The (1994)                                               4.549099
Schindler's List (1993)                                                        4.511244
Swept from the Sea (1997)                                                      4.502380
North by Northwest (1959)                                                      4.485547
High Noon (1952)                                                               4.478778
Dr. Strangelove or: How I Learne

In [0]:
list(mov_dic.values())

['Toy Story (1995)',
 'GoldenEye (1995)',
 'Four Rooms (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Twelve Monkeys (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'Richard III (1995)',
 'Seven (Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'Mighty Aphrodite (1995)',
 'Postino, Il (1994)',
 "Mr. Holland's Opus (1995)",
 'French Twist (Gazon maudit) (1995)',
 'From Dusk Till Dawn (1996)',
 'White Balloon, The (1995)',
 "Antonia's Line (1995)",
 'Angels and Insects (1995)',
 'Muppet Treasure Island (1996)',
 'Braveheart (1995)',
 'Taxi Driver (1976)',
 'Rumble in the Bronx (1995)',
 'Birdcage, The (1996)',
 'Brothers McMullen, The (1995)',
 'Bad Boys (1995)',
 'Apollo 13 (1995)',
 'Batman Forever (1995)',
 'Belle de jour (1967)',
 'Crimson Tide (1995)',
 'Crumb (1994)',
 'Desperado (1995)',
 'Doom Generation, The (1995)',
 'Free Willy 2: The Adventure Home (1995)',
 'Mad Love (1995)',
 'Nadja (1994)',
 'Net, The (1995

## SVD 

In [0]:
items.set_index("movie_id").join(ratings.set_index("movie_id"))

(100003, 26)

In [0]:
len(mov_dic)

1682

In [0]:
data = items.merge(ratings, on="movie_id", how="left")
print(data.shape)


(100003, 27)


1682

In [0]:
data["movie_id"].nunique()

1682

In [0]:
from collections import Counter

l = list(Counter(list(mov_dic.values())).values()).sort(reverse=True)

None


In [0]:
data["movie_id"].apply(lambda x: mov_dic[x])

0                                  Toy Story (1995)
1                                  Toy Story (1995)
2                                  Toy Story (1995)
3                                  Toy Story (1995)
4                                  Toy Story (1995)
5                                  Toy Story (1995)
6                                  Toy Story (1995)
7                                  Toy Story (1995)
8                                  Toy Story (1995)
9                                  Toy Story (1995)
10                                 Toy Story (1995)
11                                 Toy Story (1995)
12                                 Toy Story (1995)
13                                 Toy Story (1995)
14                                 Toy Story (1995)
15                                 Toy Story (1995)
16                                 Toy Story (1995)
17                                 Toy Story (1995)
18                                 Toy Story (1995)
19          

In [0]:
data['rating'].isnull().sum()
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,0,49,5,881250949
1,0,171,5,881250949
2,0,132,1,881250949
3,196,241,3,881250949
4,186,301,3,891717742


In [0]:
new_data = data[["movie_id", "movie title", "user_id", "rating"]]

In [0]:
new_data["movie title"].nunique()

1664

In [0]:
movie_ratings = new_data.pivot_table(index=['user_id'],columns=['movie title'],values='rating')

In [0]:
movie_ratings.shape

(944, 1664)

In [0]:
mov_dic = data.set_index("movie_id")["movie title"].to_dict()

In [0]:
df = new_data.pivot_table(index=['user_id'],columns=['movie title'],values='rating')

df.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df.shape

(944, 1664)

In [0]:
# Fill missing values with 0's
df.fillna(0, inplace=True)

# Normalize the data by converting into numpy array first
matrix = df.as_matrix()

In [0]:
user_mean = np.mean(matrix, axis=1)

In [0]:
matrix_normed = matrix - user_mean.reshape(-1,1)

In [0]:
matrix_normed.shape

(944, 1664)

In [0]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix_normed, k = 30)  # Number of latent factors is taken 50

In [0]:
sigma

array([ 59.03747832,  59.34521924,  59.42916579,  59.74979572,
        60.32680146,  60.53231162,  60.87902146,  61.17678496,
        61.58699341,  61.9461599 ,  62.11163147,  62.45454861,
        62.69007186,  63.40094328,  63.84922013,  63.92349754,
        64.75151148,  64.83621046,  65.25994307,  66.25321926,
        66.83304469,  67.1682798 ,  67.58272569,  68.5465025 ,
        68.77482557,  69.55076687,  70.33307245,  72.25432761,
        72.71257308,  73.40705137,  75.04371729,  76.14199947,
        77.78356255,  78.34822642,  80.92514917,  83.3955813 ,
        83.8873092 ,  89.76803319,  92.2861299 ,  93.69857077,
        99.57567204, 106.48564158, 111.38678301, 126.01487807,
       138.27471724, 157.66562902, 158.77018762, 217.19384938,
       244.12084822, 521.27489713])

In [0]:
''' Sigma is a diagonal matrix but with scipy it returns only those diagonal values.
So, we need to convert it back into a diagonal matrix from. 
Transformation is given below.'''

sigma = np.diag(sigma)
sigma

array([[ 59.03747832,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  59.34521924,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  59.42916579, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 217.19384938,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        244.12084822,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 521.27489713]])

In [0]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_mean.reshape(-1, 1)

In [0]:
all_user_predicted_ratings.shape

(944, 1664)

In [0]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df.columns)

In [0]:
preds_df.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
0,-0.031335,0.000821,-0.034204,-0.021158,0.001784,-0.153189,-0.083192,-0.004130,0.000049,-0.004028,...,0.003289,-0.000358,0.004787,-0.081861,0.022996,0.020154,-0.064093,-0.005055,-0.024924,0.005055
1,-0.072076,-0.060237,0.598500,2.506863,-0.078158,0.186711,1.490075,5.766853,-0.019980,0.164837,...,-0.073835,-0.193848,-0.026537,3.436720,0.514202,0.103189,0.348079,-0.027599,0.825738,0.015252
2,0.073818,0.000705,0.628568,0.443968,0.203148,0.347435,-0.250397,-1.393918,0.005043,-0.105983,...,-0.024160,0.021793,0.009585,-0.150768,-0.055185,-0.023544,0.198900,-0.007873,0.122707,0.011194
3,-0.017711,0.015344,-0.023134,0.225587,0.651014,-0.161835,-0.158951,-0.141943,0.008309,-0.008342,...,-0.014991,0.104109,-0.009795,0.083725,0.111976,-0.032097,0.101378,-0.016329,-0.021215,-0.001023
4,-0.038605,0.005756,-0.102273,-0.138900,0.369944,-0.261871,0.051986,-0.200241,0.010192,-0.089849,...,0.024312,0.078282,0.016897,0.132723,0.245203,0.102807,0.015265,0.003282,-0.014746,0.015980


In [0]:
# Recommendations for User_id = 0 in a sorted manner
preds_0 = preds_df.iloc[0,:].sort_values(ascending=False)

# Recommendations if user has not rated the movie previously
recommendations_0 = [i for i in preds_0.index if df.loc[0,i]==0]

# Recommend only top 10 items
final_recommendations_0 = recommendations_0[:10]
print("Recommendations for user with id 0 are: " + ', '.join(final_recommendations_0))

Recommendations for user with id 0 are: Return of the Jedi (1983), Raiders of the Lost Ark (1981), Indiana Jones and the Last Crusade (1989), Godfather, The (1972), Princess Bride, The (1987), Back to the Future (1985), Blade Runner (1982), Toy Story (1995), Terminator, The (1984), Pulp Fiction (1994)


In [0]:
np.dot(U, V.T).shape

(944, 1682)

In [0]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,0,49,5,881250949
1,0,171,5,881250949
2,0,132,1,881250949
3,196,241,3,881250949
4,186,301,3,891717742


## tip

In [0]:
data = items.merge(ratings, on="movie_id", how="left")
print(data.shape)

(100003, 27)


In [0]:
new_data = data[["movie_id", "movie title", "user_id", "rating"]]

In [0]:
new_data["movie title"].nunique()

1664

In [0]:
movie_ratings = new_data.pivot_table(index=['user_id'],columns=['movie title'],values='rating')

In [0]:
movie_ratings.shape

(944, 1664)

In [0]:
df = pd.DataFrame(np.zeros((944, 1682)))

In [0]:
ratings.iloc[0,:]

user_id                   0
movie_id                 49
rating                    5
unix_timestamp    881250949
Name: 0, dtype: int64

In [0]:
for i in range(len(ratings)):
  rat = ratings.iloc[i,:]
  df.iloc[rat[0],rat[1]] = rat[2]

In [0]:
R_actual = df

In [0]:
R_actual.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
R_pred = pd.DataFrame(R)

In [0]:
R_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.665727,3.235047,2.968430,3.355351,3.166419,3.548365,3.607330,3.766282,3.591586,3.565437,...,3.100065,3.446041,3.116519,2.813100,3.284824,2.730119,3.221462,2.972895,3.291633,3.235823
1,4.130785,3.100493,2.740543,3.676160,2.920782,4.110650,4.159802,4.325611,4.408041,4.238366,...,3.275114,3.853290,3.278053,2.748811,3.552240,2.675717,3.483985,3.047647,3.546170,3.449395
2,4.195002,3.215589,2.645116,3.567852,3.113731,4.007411,4.001553,4.288305,4.113411,3.835725,...,3.289668,3.818775,3.294399,2.798412,3.546478,2.732283,3.475404,3.082632,3.538525,3.456376
3,3.113927,2.403545,2.394650,3.122022,2.377880,3.109819,3.304140,3.333372,3.682083,3.491730,...,2.934493,3.179289,2.912877,2.700490,3.013641,2.470748,2.975347,2.700698,3.021872,2.963229
4,4.718051,4.472553,4.327277,4.304169,4.057276,4.999886,4.688836,4.702441,4.904922,4.938152,...,3.762127,4.367909,3.822663,3.326610,4.132588,3.461342,4.056078,3.761342,4.146522,4.047360


In [0]:
# Recommendations for User_id = 0 in a sorted manner
preds_0 = R_pred.iloc[5,:].sort_values(ascending=False)


# Recommendations if user has not rated the movie previously
recommendations_0 = [i for i in preds_0.index if R_actual.loc[5,i]==0]

# Recommend only top 10 items
final_recommendations_0 = recommendations_0[:10]
final_recommendations_0 = [mov_dic[i] for i in final_recommendations_0]
print(final_recommendations_0)
# print("Recommendations for user with id 0 are: " + ', '.join(final_recommendations_0))

['Pather Panchali (1955)', 'Hugo Pool (1997)', 'Casablanca (1942)', 'Swept from the Sea (1997)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Citizen Kane (1941)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 'Third Man, The (1949)', 'Godfather, The (1972)', 'Rear Window (1954)']
